# # Classifiers comparison on texts with naive Bayes assumption

In this session of laboratory we compare two models for categorical data probabilistic modeling: 
1. multivariate Bernoulli 
2. multinomial on a dataset 

We adopt a dataset on Twitter messages labelled with emotions (Joy vs Sadness).

The following program shows the loading of the data from a file.

Data are loaded into a matrix X adopting a sparse matrix representation, in order to save space and time.
Sparse matrix representation (in the csr format) represents in three "parallel" arrays the value of the matrix cells that are different from zero and the indices of those matrix cells.
The arrays are called: 
- data
- row
- col

- data[i] stores the value of the matrix cell #i whose indexes are contained in row[i] and col[i] 
- row[i] stores the index of the row in the matrix of the cell #i, 
- col[i] stores the index of the column of the cell #i.


The data file is in csv format.
Any Twitter message has been preprocessed by a Natural Language pipeline which eliminated stop words and substituted the interesting document elements with an integer identifier.  
The interesting document elements might be words, emoji or emoticons. The elements could be repeated in the same document and are uniquely identified in the documents by the same integer number (named "element_id" in the program). This "element_id" number will be used as the index of the column of the data matrix, for the purposes of storage of data.

Each row of the CSV file reports the content of a document (a Twitter message).It is formed as a list of integer number pairs, followed by a string which is the label of the document ("Joy" or "Sadness").
The first number of the pair is the identifier of a document element (the "element_id"); 
the second number of the pair is the count (frequency) of that element in that document.

The dataset has:

tot_n_docs (or rows in the file) =n_rows=11981

n_features (total number of distinct words in the corpus)=11288



The following program reads the data file and loads in a sparse way the matrix using the scipy.sparse library

In [2]:

import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score

class_labels = ["Joy","Sadness"]
n_features=11288 # number of columns in the matrix = number of features (distinct elements in the documents)
n_rows=11981 # number rows of the matrix
n_elements=71474 # number of the existing values in the matrix (not empty, to be loaded in the matrix in a sparse way)

#path_training="/Users/meo/Documents/Didattica/Laboratorio-15-16-Jupyter/"
path_training="./"
file_name="joy_sadness6000.txt"

# declare the row and col arrays with the indexes of the matrix cells (non empty) to be loaded from file
# they are needed because the matrix is sparse and we load in the matrix only the elements which are present
row=np.empty(n_elements, dtype=int)
col=np.empty(n_elements, dtype=int)
data=np.empty(n_elements, dtype=int)

row_n=0 # number of current row to be read and managed
cur_el=0 # position in the three arrays: row, col and data
twitter_labels=[] # list of class labels (target array) of the documents (twitter) that will be read from the input file
twitter_target=[] # list of 0/1 for class labels
with open(path_training + file_name, "r") as fi:
    for line in fi:
        el_list=line.split(',')  # list of integers read from a row of the file
        l=len(el_list)
        last_el=el_list[l-1] # I grab the last element in the list which is the class label
        class_name=last_el.strip() # eliminate the '\n'
        twitter_labels.append(class_name)
        # twitter_labels contains the labels (Joy/Sadness); twitter_target contains 0/1 for the respective labels
        if (class_name==class_labels[0]):
           twitter_target.append(0)
        else:
           twitter_target.append(1)
        i=0 # I start reading all the doc elements from the beginning of the list
        while(i<(l-1)):
            element_id=int(el_list[i]) # identifier of the element in the document equivalent to the column index
            element_id=element_id-1 # the index starts from 0 (the read id starts from 1)
            i=i+1
            value_cell=int(el_list[i]) # make access to the following value in the file which is the count of the element in the documento 
            i=i+1
            row[cur_el]=row_n # load the data in the three arrays: the first two are the row and col indexes; the last one is the matrix cell value
            col[cur_el]=element_id
            data[cur_el]=value_cell
            cur_el=cur_el+1
        row_n=row_n+1
fi.close
print("final n_row="+str(row))
# loads the matrix by means of the indexes and the values in the three arrays just filled
twitter_data=csr_matrix((data, (row, col)), shape=(n_rows, n_features)).toarray()
print("resulting matrix:")
print(twitter_data)
print(twitter_labels)
print(twitter_target)


final n_row=[0 0 0 ... 0 0 0]
resulting matrix:
[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
['Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy1', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy

Write a program in the following cell that splits the data matrix in training and test set (by random selection) and predicts the class (Joy/Sadness) of the messages on the basis of the words. 
Consider the two possible models:
multivariate Bernoulli and multinomial Bernoulli.
Find the accuracy of the models and test is the observed differences are significant.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(twitter_data, twitter_target, test_size=0.3)

# Multivariates

In [4]:
import numpy as np

# Aggiungo pseudoconteggi
X_train = np.vstack([
    X_train,
    np.ones((1, X_train.shape[1])),
    np.ones((1, X_train.shape[1])),
    np.zeros((1, X_train.shape[1])),
    np.zeros((1, X_train.shape[1]))
])

y_train.append(0)
y_train.append(1)
y_train.append(0)
y_train.append(1)

In [5]:
#converto i numeri != 0 in 1
X_train_multivariates = (X_train != 0).astype(int)

#matrice con joy a 1
joy_X_multivariate = X_train_multivariates * (1 - np.array(y_train).reshape(-1, 1))
#matrice con i sadness a 1
sadness_X_multivariate = X_train_multivariates * np.array(y_train).reshape(-1, 1)


#sommo le colonne
count_multivariates_joy = np.sum(joy_X_multivariate, axis=0)
count_multivariates_sadness = np.sum(sadness_X_multivariate, axis=0)

#divido per il numero di esempi
count_multivariates_joy = count_multivariates_joy.astype(float) / np.sum((1 - np.array(y_train).reshape(-1, 1)))
count_multivariates_sadness = count_multivariates_sadness.astype(float) / np.sum(np.array(y_train).reshape(-1, 1))

Costruisco il modello

In [6]:
def likelihood(example, prob_estimate):
    f = lambda xi, theta_i: theta_i ** xi * (1 - theta_i) ** (1 - xi)
    res = [f(example[i], prob_estimate[i]) for i in range(len(example))]
    return np.prod(res)

def max_likelihood(example):
    joy_likelihood = likelihood(example, count_multivariates_joy)
    sadness_likelihood = likelihood(example, count_multivariates_sadness)

    if joy_likelihood > sadness_likelihood:
        return 0 #joy
    else:
        return 1 #sadness

In [7]:
X_test_multivariates = (X_test != 0).astype(int)

test_result_multivariate = [max_likelihood(test_i) for test_i in X_test_multivariates]

In [18]:
accuracy_score(test_result_multivariate,y_test)

0.9499304589707928

# Multinomiale

In [9]:
import numpy as np

# Aggiungo pseudoconteggi
X_train = np.vstack([
    X_train,
    np.ones((1, X_train.shape[1])),
    np.ones((1, X_train.shape[1])),
    np.zeros((1, X_train.shape[1])),
    np.zeros((1, X_train.shape[1]))
])

y_train.append(0)
y_train.append(1)
y_train.append(0)
y_train.append(1)

In [10]:
#matrice con joy a 1
joy_X_multinomiale = X_train * (1 - np.array(y_train).reshape(-1, 1))
#matrice con i sadness a 1
sadness_X_multinomiale = X_train * np.array(y_train).reshape(-1, 1)


#sommo le colonne
count_multinomiale_joy = np.sum(joy_X_multinomiale, axis=0)
count_multinomiale_sadness = np.sum(sadness_X_multinomiale, axis=0)

#divido per il numero di esempi
count_multinomiale_joy = count_multinomiale_joy.astype(float) / np.sum(count_multinomiale_joy)
count_multinomiale_sadness = count_multinomiale_sadness.astype(float) / np.sum(count_multinomiale_sadness)

Costruisco il modello

In [11]:
import math


def likelihood_multinomial(example, prob_estimate):
    f = lambda xi, theta_i: theta_i ** xi / math.factorial(xi)
    res = [f(example[i], prob_estimate[i]) for i in range(len(example))]
    coeff = math.factorial(np.sum(example))
    return coeff * np.prod(res)

def max_likelihood_multinomial(example):
    joy_likelihood = likelihood(example, count_multinomiale_joy)
    sadness_likelihood = likelihood(example, count_multinomiale_sadness)

    if joy_likelihood > sadness_likelihood:
        return 0 #joy
    else:
        return 1 #sadness

In [12]:
test_result_multinomiale = [max_likelihood_multinomial(test_i) for test_i in X_test]

In [13]:
accuracy_score(test_result_multinomiale,y_test)

0.9463143254520167

# TEST statistico tra cl multinomiale e multivariato

In [27]:
from scipy.stats import ttest_ind

multivariate_res = [test_result_multivariate[i] ^ y_test[i] for i in range(len(test_result_multivariate))]
multinomiale_res = [test_result_multinomiale[i] ^ y_test[i] for i in range(len(test_result_multinomiale))]



t_stat, p_value = ttest_ind(multinomiale_res,multivariate_res)
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

T-statistic: 0.6912111839563995
P-value: 0.48945515850473165


Dunque i due classificatori hanno risultati uguali